In [1]:
"""
This notebook will allow you to run a model on a single prepared dataset. 
"""
import torch
import numpy as np
import torch.optim as optim
from torch.utils import data
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import pandas as pd
from sklearn.metrics import roc_auc_score
import datetime
import matplotlib.animation as animation
import matplotlib.pyplot as plt
from io import StringIO
import ffmpeg
import pdb
from functions import create_pytorch_dataset
from functions import get_window_metrics
from functions import get_frame_metrics
from functions import animate
import scipy.signal

In [2]:
# Lets load the H%PY dataset into a pytorch dataset class.Please see 
# dataset_creator on how to generate the H5PY file. 

# Name of the H5PY dataset 
dset = "Edits/Thermal" #where the orginal data is stored 
name = "Thermal_Edit" # name of the h5py file
path = "H5Data\Data_set-{}-imgdim64x64.h5".format(name) # location of the h5py file
# this will also window the data at a set size, and with the set stride 

window_len = 8
stride = 1

Test_Dataset, test_dataloader, Train_Dataset, train_dataloader = create_pytorch_dataset(name, dset, path, window_len, stride)



['Fall0', 'Fall1', 'Fall10', 'Fall100', 'Fall101', 'Fall102', 'Fall103', 'Fall104', 'Fall105', 'Fall106', 'Fall107', 'Fall108', 'Fall109', 'Fall11', 'Fall110', 'Fall111', 'Fall112', 'Fall113', 'Fall114', 'Fall115', 'Fall116', 'Fall117', 'Fall118', 'Fall119', 'Fall12', 'Fall120', 'Fall121', 'Fall122', 'Fall123', 'Fall124', 'Fall125', 'Fall126', 'Fall127', 'Fall128', 'Fall129', 'Fall13', 'Fall130', 'Fall131', 'Fall132', 'Fall133', 'Fall134', 'Fall135', 'Fall136', 'Fall137', 'Fall138', 'Fall139', 'Fall14', 'Fall15', 'Fall150', 'Fall151', 'Fall152', 'Fall153', 'Fall154', 'Fall155', 'Fall156', 'Fall157', 'Fall158', 'Fall159', 'Fall16', 'Fall160', 'Fall161', 'Fall162', 'Fall163', 'Fall164', 'Fall165', 'Fall166', 'Fall167', 'Fall168', 'Fall169', 'Fall17', 'Fall170', 'Fall171', 'Fall172', 'Fall173', 'Fall174', 'Fall175', 'Fall176', 'Fall177', 'Fall178', 'Fall179', 'Fall18', 'Fall180', 'Fall181', 'Fall182', 'Fall183', 'Fall184', 'Fall185', 'Fall186', 'Fall187', 'Fall188', 'Fall189', 'Fall19', '

In [3]:

# Now that we have our datasets prepared, lets write our model 

class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        # first layer
        self.ec1 = nn.Conv3d(1, 16, (5, 3, 3), stride=1, padding=(2, 1, 1),)
        self.em1 = nn.MaxPool3d((1, 2, 2), return_indices=True)
        #self.ed1 = nn.Dropout3d(p=0.25)
        # second layer
        self.ec2 = nn.Conv3d(16, 8, (5, 3, 3), stride=1, padding=(2, 1, 1))
        self.em2 = nn.MaxPool3d((2, 2, 2), return_indices=True)
        #self.ed2 = nn.Dropout3d(p=0.25)
        # third layer
        self.ec3 = nn.Conv3d(8, 8, (5, 3, 3), stride=1, padding=(2, 1, 1))
        self.em3 = nn.MaxPool3d((2, 2, 2), return_indices=True)
        # encoding done, time to decode
        self.dc1 = nn.ConvTranspose3d(8, 8, (5, 3, 3), stride=1, padding=(2, 1, 1))
        self.dm1 = nn.MaxUnpool3d((2, 2, 2))
        # inverse of 2nd Conv
        self.dc2 = nn.ConvTranspose3d(8, 8, (5, 3, 3), stride=1, padding=(2, 1, 1))
        self.dm2 = nn.MaxUnpool3d((2, 2, 2))
        # inverse of 1st Conv
        self.dc3 = nn.ConvTranspose3d(8, 16, (5, 3, 3), stride=1, padding=(2, 1, 1))
        self.dm3 = nn.MaxUnpool3d((1, 2, 2))
        # final inverse
        self.dc4 = nn.ConvTranspose3d(16, 1, (5, 3, 3), stride=1, padding=(2, 1, 1))

    def forward(self, x):
        # *** start of encoder
        x = x.permute(1, 0, 2, 3, 4)  # reorder to have correct dimensions
        # (batch_size, chanels, depth, width, height)
        _ec1 = F.relu(self.ec1(x))
        _em1, i1 = self.em1(_ec1)
        #_ec1 = self.ed1(_ec1)
        # second layer
        _ec2 = F.relu(self.ec2(_em1))
        _em2, i2 = self.em2(_ec2)
        #_em2 = self.ed2(_em2)
        # third layer
        _ec3 = F.relu(self.ec3(_em2))
        _em3, i3 = self.em3(_ec3)
        # print("====== Encoding Done =========")
        # *** encoding done, time to decode
        _dc1 = F.relu(self.dc1(_em3))
        _dm1 = self.dm1(_dc1, i3, output_size=_em2.size())
        # second layer
        _dc2 = F.relu(self.dc2(_dm1))
        _dm2 = self.dm2(_dc2, i2)
        # third layer
        _dc3 = F.relu(self.dc3(_dm2))
        _dm3 = self.dm3(_dc3, i1)

        re_x = torch.tanh(self.dc4(_dm3))
        return re_x


In [4]:
# Now lets train our model

# prepare for GPU training 
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
torch.cuda.empty_cache()

# and lets set the hyperparameters! 

dropout = 0.25
learning_rate = 0.0002
num_epochs = 20
chunk_size = 128
forward_chunk = 8 
forward_chunk_size = 8 # this is smaller due to memory constrains 
w1 = 1
w2 = 0.00001

# select which model - you could load your own or put it in the function above 
model = Autoencoder().to(device)
loss_fn = nn.MSELoss()
def spatial_temporal_loss(input):
    loss_kernal = [
    [[ 0.,  -1.,  0.], 
    [ -1.,  -1.,  -1.],
    [ 0.,  -1.,  0.]], 

    [[ 0.,  -1.,  0.], 
    [ -1.,  14.,  -1.],
    [ 0.,  -1.,  0.]], 

    [[ 0.,  -1.,  0.], 
    [ -1.,  -1.,  -1.],
    [ 0.,  -1.,  0.]] ]
    loss_kernal = np.array(loss_kernal)
    for i in range(input.shape[1]):
        image = input[0,i,:,:,:].data.cpu().numpy()
        convlved = scipy.signal.fftconvolve(image, loss_kernal, mode='same')
        adjactent_loss = sum(sum(sum(convlved)))
    return(adjactent_loss)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


cuda


In [5]:
def train_model(filepath):
    model.train()
    for epoch in range(num_epochs):
        val_loss = 0
        for i, (sample, labels) in enumerate(train_dataloader):
            # ===================forward=====================
            sample = sample.to(device, dtype=torch.float)
            # split sample into smaller sizes due to GPU memory constraints
            chunks = torch.split(sample, chunk_size, dim=1)
            for chunk in chunks:
                output = model(chunk)
                output = output.to(device).permute(1, 0, 2, 3, 4)
                model.zero_grad()
                loss = w1*loss_fn(output, chunk) + w2*abs((spatial_temporal_loss(output) - spatial_temporal_loss(chunk)))
                # ===================backward====================
                # Getting gradients w.r.t. parameters
                loss.backward()
                # Updating parameters
                optimizer.step()
                # Clear gradients w.r.t. parameters
                optimizer.zero_grad()
                torch.cuda.empty_cache()

        # ===================log========================
        print("epoch [{}/{}], loss:{:.4f}".format(epoch + 1, num_epochs, loss.item()))
        torch.save(model.state_dict(), filepath) # save the model each epoch at location filepath
        
    torch.cuda.empty_cache()

In [6]:

def foward_pass(path):
    model.load_state_dict(torch.load(path)) # load a saved model 
    model.eval()
    frame_stats = [] 
    window_stats = [] 

    with torch.no_grad():
        print("foward pass occuring")
        # just forward pass of model on test dataset
        for j, (sample, labels) in enumerate(test_dataloader):
            print(j)
            # foward pass to get output
            torch.cuda.empty_cache()
            sample = sample.to(device, dtype=torch.float)
            chunks = torch.split(sample, forward_chunk, dim=1)
            recon_vid = []
            for chunk in chunks:
                output = model(chunk)
                output = output.to(device).permute(1, 0, 2, 3, 4)
                recon_vid.append(output)
                torch.cuda.empty_cache()
   
            output = torch.cat(recon_vid, dim=1)

            # convert tensors to numpy arrays for easy manipluations
            sample = sample.data.cpu().numpy()
            output = output.data.cpu().numpy()
            labels = labels.data.cpu().numpy()

            frame_mean, frame_std, frame_labels = get_frame_metrics(output, sample, labels, window_len)
            mean_window_error, std_window_error, window_labels = get_window_metrics(output, sample, labels, window_len)
            frame_stats.append([frame_mean, frame_std, frame_labels])
            window_stats.append([mean_window_error, std_window_error, window_labels])

             if j % 50 == 0:
                animate(sample[0, :, :, :, :], output[0, :, :, :, :], frame_mean, dset, start_time)
        
    return(frame_stats, window_stats)


            

In [7]:


start_time = str(datetime.datetime.today().strftime("%Y-%m-%d-%H-%M-%S"))
print(start_time)
filepath = (
    "Models\\"
    + dset 
    + 'GradientLoss'
    + start_time
)
filepath = 'Models\Edits\IP2020-12-03-12-40-18'
print(filepath)
#train_model(filepath) 
import functions
from functions import animate


frame_stats, window_stats = foward_pass(filepath)


2021-01-07-16-11-38
Models\Edits/ThermalGradientLoss2021-01-07-16-11-38
epoch [1/20], loss:0.0041
epoch [2/20], loss:0.0027
epoch [3/20], loss:0.0035
epoch [4/20], loss:0.0032
epoch [5/20], loss:0.0029
epoch [6/20], loss:0.0031
epoch [7/20], loss:0.0029
epoch [8/20], loss:0.0025
epoch [9/20], loss:0.0025
epoch [10/20], loss:0.0024
epoch [11/20], loss:0.0020
epoch [12/20], loss:0.0020
epoch [13/20], loss:0.0023
epoch [14/20], loss:0.0022
epoch [15/20], loss:0.0024
epoch [16/20], loss:0.0024
epoch [17/20], loss:0.0024
epoch [18/20], loss:0.0028
epoch [19/20], loss:0.0030
epoch [20/20], loss:0.0030
foward pass occuring
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
12

In [8]:
import functions
from functions import get_total_performance_metrics
from importlib import reload
reload(functions)
from functions import get_total_performance_metrics

get_total_performance_metrics(frame_stats, window_stats, window_len)


#get_total_performance_metrics(originals, reconstruced, testing_labels, window_len)

(187, 5, 8)
saving
